# Digit Recognizer Using Keras 

Download dataset from <a href="https://www.kaggle.com/c/digit-recognizer/data"> kaggle </a> and extract into current working directory

In [1]:
import os
import numpy as np
import pandas as pd
import scipy.stats
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

D:\AI\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_mnist(directory, train=True):    
    if train:
        data = np.genfromtxt(directory + r'train.csv', delimiter=',',
                             dtype='float32', skip_header=1)
        
        # First column of train.csv is label then pixel 0-783
        labels = data[:, 0]
        data = data[:, 1:]        

        # Pixel data must be reshaped to 2D array
        data = np.reshape(data, (data.shape[0], 28, 28))
        
        # One-hot encode labels
        labels = to_categorical(labels, 10)     
        
        return data, labels
    
    # Note: test.csv has no label column    
    else:
        data = np.genfromtxt(directory + r'test.csv', delimiter=',',
                             dtype='float32', skip_header=1)
        data = np.reshape(data, (data.shape[0], 28, 28))
        return data

In [3]:
def preprocess_images(data):
    return data / 255.0

In [4]:
def mnist_cnn(input_shape=(28, 28, 1), num_classes=10, lr=1E-3, dropout_rate=0.25):
    model = Sequential()
    model.add(Conv2D(64, (5, 5), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, (5, 5), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=lr, decay=1e-6),
                  metrics=['accuracy'])
    model.summary()
    return model

In [5]:
# Create model
model = mnist_cnn(lr=1E-3, dropout_rate=0.4)

# Setup data augmenter
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=2/28,
    height_shift_range=2/28)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 64)        1664      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 128)         512       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
__________

In [6]:
# This should be changed to the directory of your datasets
directory = r'data/'

# Load and normalize train set
X_train, Y_train = load_mnist(directory, train=True)
X_train = preprocess_images(X_train)
X_train = np.expand_dims(X_train, axis=-1)

# Load and normalize test set
X_test = load_mnist(directory, train=False)
X_test = preprocess_images(X_test)
X_test = np.expand_dims(X_test, axis=-1)

print('Image Dimensions:', X_train.shape[1:])
print('Number of Training Samples:', X_train.shape[0])
print('Number of Test Samples:', X_test.shape[0])

Image Dimensions: (28, 28, 1)
Number of Training Samples: 42000
Number of Test Samples: 28000


In [7]:
models = []
for i in range(2):
    lr_anneal = ReduceLROnPlateau(monitor='loss', patience=3, factor=0.1)
    model.fit_generator(datagen.flow(X_train, Y_train, batch_size=32),
                        epochs=3, steps_per_epoch=len(X_train) / 64,
                        callbacks=[lr_anneal])
    models.append(model)

Epoch 1/3
657/656 [==============================] - 265s 403ms/step - loss: 0.2842 - acc: 0.9155
Epoch 2/3
657/656 [==============================] - 257s 391ms/step - loss: 0.1309 - acc: 0.9587
Epoch 3/3
657/656 [==============================] - 254s 387ms/step - loss: 0.1032 - acc: 0.9696
Epoch 1/3
657/656 [==============================] - 255s 387ms/step - loss: 0.0922 - acc: 0.9714
Epoch 2/3
657/656 [==============================] - 246s 374ms/step - loss: 0.0860 - acc: 0.9734
Epoch 3/3
657/656 [==============================] - 248s 378ms/step - loss: 0.0810 - acc: 0.9758


In [8]:
# Predict labels using each model
labels = []
for model in models:
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    labels.append(y_pred)
    
# Perform ensemble classification voting
labels = np.array(labels)
labels = np.transpose(labels, (1, 0))
labels = scipy.stats.mode(labels, axis=-1)[0]
labels = np.squeeze(labels)

In [11]:
labels

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [12]:
# Save this model

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [16]:
#Load the model for production
from keras.models import model_from_json
import tensorflow as tf
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded Model from disk")
#compile and evaluate loaded model
lr=1E-3
loaded_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=lr, decay=1e-6),metrics=['accuracy'])
graph = tf.get_default_graph()

Loaded Model from disk


In [105]:
from PIL import Image, ImageFilter


def imageprepare(argv):
    """
    This function returns the pixel values.
    The imput is a png file location.
    """
    im = Image.open(argv).convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    newImage = Image.new('L', (28, 28), (255))  # creates white canvas of 28x28 pixels

    if width > height:  # check which dimension is bigger
        # Width is bigger. Width becomes 20 pixels.
        nheight = int(round((20.0 / width * height), 0))  # resize height according to ratio width
        if (nheight == 0):  # rare case but minimum is 1 pixel
            nheight = 1
            # resize and sharpen
        img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((28 - nheight) / 2), 0))  # calculate horizontal position
        newImage.paste(img, (4, wtop))  # paste resized image on white canvas
    else:
        # Height is bigger. Heigth becomes 20 pixels.
        nwidth = int(round((20.0 / height * width), 0))  # resize width according to ratio height
        if (nwidth == 0):  # rare case but minimum is 1 pixel
            nwidth = 1
            # resize and sharpen
        img = im.resize((nwidth, 20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((28 - nwidth) / 2), 0))  # caculate vertical pozition
        newImage.paste(img, (wleft, 4))  # paste resized image on white canvas
    #newImage.save("sample.png")

    tv = list(newImage.getdata())  # get pixel values

    # normalize pixels to 0 and 1. 0 is pure white, 1 is pure black.
    tva = [(255 - x) * 1.0 / 255.0 for x in tv]
    
    
    return tva


def predict():
    x=imageprepare('./img1.png')#file path here
    x=np.asarray(x).reshape(1,28,28,1)
    out = loaded_model.predict(x)
    #print(out)
    print(np.argmax(out, axis=1))



In [106]:
predict()

[4]
